## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference. (**Done**)
    * Extract out professors who do not teach
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen:
        * No professors teach two courses back-to-back


In [2]:
import pandas as pd
from docplex.mp.model import Model
import math
import random

### Data Exploration and Cleaning

In [5]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)
print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
display(file_sheets[0], classrooms_df.head(5))

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
display(file_sheets[2], course_catalog_df.head(5))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
display(file_sheets[3], courses_offered.head(45))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
display(file_sheets[5], qualifications_df.head(5))

# For reproducibility
random.seed(1345)

Sheets : ['Classrooms', 'Colleges and Departments', 'Course Catalog', 'Courses Offered Fall 2021', 'Professors', 'Qualification']


'Classrooms'

,Building,Building Code,Classroom,Capacity
0,Durland,DU,1029,47
1,Durland,DU,1041,24
2,Durland,DU,1052,60
3,Durland,DU,1061,47
4,Durland,DU,1063,47


'Colleges and Departments'

,College,College Code,Department,Department Code
0,College of Agriculture,CAGRI,Entomology,ENTOM


...

,Department Code,Course Number,Course Name,Sections,Day,Max Enrollement
0,ENTOM,300,Economic Entomology,1,M/W/F,50
1,ENTOM,301,Insects and People,3,TU/TH,50
2,ENTOM,305,Animal Health Entomology,3,TU/TH,50
3,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50
4,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50
5,ENTOM,589,Turfgrass Insects and Their Management,1,M/W/F,40
6,ENTOM,602,Forensic Entomology,1,TU/TH,40
7,ENTOM,621,Introduction to Biological Control,1,TU/TH,40
8,ENTOM,625,Introductory Integrative Behavioral Ecology,1,TU/TH,40
9,ENTOM,635,Insect Evolution,1,TU/TH,40


'Professors'

,Department Code,Faculty Name,Workload Credit Hours
0,ENTOM,Frank H. Arthur,6
1,ENTOM,James F. Campbell,9
2,ENTOM,Ming-Shun Chen,9
3,ENTOM,Raymond Cloyd,9
4,ENTOM,Lee Cohnstaedt,9


'Qualification'

,Department Code,Faculty Name,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,ENTOM 621,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


### Functions and Customizations

In [16]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    ''''''
    def get_day_name(day:str):
        days_name = {'M':'Monday', 'TU':'Tuesday', 'W':'Wednesday', 'TH':'Thursday', 'F':'Friday'}
        return days_name[day]
    
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Section Number'})
    result = result.sort_values(by=['Course Number', 'Section Number']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department Code', 'Course Number', 'Course Name'])
    
    courses_offered = result
    result = pd.DataFrame()
    
    for course in courses_offered.itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        days_taught = course['Day'][0]
        days_taught = days_taught.split('/')
        for i in days_taught:
            course['Day'] = i
            result = result.append([course], ignore_index=True)
            
    result['Day'] = result.apply(lambda row : get_day_name(row.Day), axis=1)
    return result

standardize_courses_offered(courses_offered, course_catalog_df)

,Department Code,Course Number,Course Name,Section Number,Day,Max Enrollement,Credit Hours
0,ENTOM,300,Economic Entomology,1,Monday,50,3
1,ENTOM,300,Economic Entomology,1,Wednesday,50,3
2,ENTOM,300,Economic Entomology,1,Friday,50,3
3,ENTOM,301,Insects and People,1,Tuesday,50,3
4,ENTOM,301,Insects and People,1,Thursday,50,3
...,...,...,...,...,...,...,...
87,ENTOM,880,Ecological Genomics,1,Tuesday,20,5
88,ENTOM,880,Ecological Genomics,1,Wednesday,20,5
89,ENTOM,880,Ecological Genomics,1,Thursday,20,5
90,ENTOM,885,Conventional and Molecular Methods for Evaluat...,1,Monday,20,3


In [7]:
## Clean
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload Credit Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            
        #display(preferences)

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty Name', 'Department Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df.head(10))

,Department Code,Faculty Name,Workload Credit Hours,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,6,0,2,8,0,5,0,0,...,3,0,0,4,1,0,0,0,0,0
1,ENTOM,James F. Campbell,9,0,6,8,0,4,1,0,...,0,0,0,0,3,0,0,0,7,5
2,ENTOM,Ming-Shun Chen,9,0,6,2,3,0,8,0,...,0,0,0,0,0,0,0,1,7,5
3,ENTOM,Raymond Cloyd,9,0,4,11,3,2,8,0,...,0,9,0,0,10,7,0,0,0,0
4,ENTOM,Lee Cohnstaedt,9,6,3,4,5,10,0,2,...,11,0,0,0,7,0,9,0,0,0
5,ENTOM,Srinivas Kambhampati,6,0,0,6,3,0,0,0,...,0,0,0,9,0,0,7,0,8,0
6,ENTOM,Tania Kim,3,7,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ENTOM,Berlin Luxelly Londono Renteria,6,4,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,ENTOM,Jeremy L. Marshall,9,7,3,2,6,0,0,8,...,0,0,0,0,0,0,0,0,0,0
9,ENTOM,Brian P. McCornack,9,0,0,0,1,0,0,0,...,3,2,0,5,7,0,0,0,0,0


In [24]:
def get_period_df() -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday']
    days_of_weeks = dict(zip(days, range(1, 6)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_hour = 8
    end_hour = 18
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 21)) * 5, 'Day Name' : day_names, 'Day Number' : day_digits, 
                                   'Start Time' : start_time, 'End Time' : end_time})

periods_df = get_period_df()

In [29]:
def get_column_value_as_list(df:pd.DataFrame, colname:str) -> pd.DataFrame:
    '''Get the values of a data frame column as a Python list'''
    if colname in df.columns:
        return df[colname].values.tolist()
    else:
        return None

# How to use
get_column_value_as_list(professors_df, 'Faculty Name')
get_column_value_as_list(periods_df, 'Day Name')

['Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Monday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Tuesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Wednesday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',
 'Thrusday',


### Mathematical Model

In [150]:
## Model Environment
model = Model(name='Course Scheduling')
model.print_information()

## Decision Variables

## Constraints

    #Need
        #Capacity
        #Time start no earlier than 0800
        #Time end no later than 1800
        #Every course must be assigned
        #no section  can  be  assigned  to  more  than  one  professor

    #Rule 1
        #Prof doesn't have back to back course
            #Do by sum of consecutive periods with a break of some arbitrary number
    #Rule 2
        #No scheduled classes after noon on Friday



## Objective Function

#total_preferance = sum(preferance_df )

#mdl.maximize(total_preferance)

Model: Course Scheduling
 - number of variables: 0
   - binary=0, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP
